In [1]:
# Import packages
import pandas as pd 
import numpy as np 
import geopandas as gpd

from tqdm import tqdm
import requests
import io
import json
import plotly.express as px


## Step 1: Select all 'wijken' (neighborhoods) of Utrecht City

In [2]:
neighborhoods = gpd.read_file("../data/external/WijkBuurtkaart_2020_v1/buurt_2020_v1.shp")
utrecht_neigh = neighborhoods[['BU_NAAM','GM_NAAM','BEV_DICHTH','AANT_INW', 'AANT_MAN', 'AANT_VROUW','AANTAL_HH','OPP_TOT', 'OPP_LAND', 'OPP_WATER','geometry']][neighborhoods["GM_NAAM"]=="Utrecht"]
utrecht_neigh

,BU_NAAM,GM_NAAM,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,AANTAL_HH,OPP_TOT,OPP_LAND,OPP_WATER,geometry
3777,"Welgelegen, Den Hommel",Utrecht,4078,1465,705,760,885,39,36,3,"POLYGON ((135006.339 455419.888, 135197.876 45..."
3778,Oog in Al,Utrecht,9859,4345,2165,2175,1535,47,44,3,"POLYGON ((134605.713 455754.201, 134652.927 45..."
3779,Halve Maan-Zuid,Utrecht,8148,1575,830,745,555,23,19,4,"POLYGON ((134072.101 455635.056, 134074.419 45..."
3780,Halve Maan-Noord,Utrecht,9336,1735,830,900,870,25,19,6,"POLYGON ((133770.466 456647.956, 133786.743 45..."
3781,Lombok-Oost,Utrecht,17630,2285,1110,1175,1470,13,13,0,"POLYGON ((135822.953 456037.558, 135715.534 45..."
...,...,...,...,...,...,...,...,...,...,...,...
3883,Veldhuizen,Utrecht,5631,9330,4635,4690,3460,189,166,23,"POLYGON ((129692.900 455386.278, 129786.270 45..."
3884,De Meern-Noord,Utrecht,3925,5735,2875,2860,2080,148,146,2,"POLYGON ((130301.248 456100.621, 130314.591 45..."
3885,De Meern-Zuid,Utrecht,1671,6425,3160,3260,2715,389,385,5,"POLYGON ((130607.044 455121.579, 130987.916 45..."
3886,Bedrijvengebied Oudenrijn,Utrecht,170,265,125,140,115,163,158,5,"POLYGON ((132114.247 455104.252, 132112.341 45..."


## Step 2: Add "centroid" as a measurement station to each neighborhood

In [3]:
utrecht_neigh["measurement_station"] = utrecht_neigh.representative_point()

In [4]:
utrecht_neigh

,BU_NAAM,GM_NAAM,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,AANTAL_HH,OPP_TOT,OPP_LAND,OPP_WATER,geometry,measurement_station
3777,"Welgelegen, Den Hommel",Utrecht,4078,1465,705,760,885,39,36,3,"POLYGON ((135006.339 455419.888, 135197.876 45...",POINT (134783.298 455066.413)
3778,Oog in Al,Utrecht,9859,4345,2165,2175,1535,47,44,3,"POLYGON ((134605.713 455754.201, 134652.927 45...",POINT (134526.708 455515.178)
3779,Halve Maan-Zuid,Utrecht,8148,1575,830,745,555,23,19,4,"POLYGON ((134072.101 455635.056, 134074.419 45...",POINT (134062.946 455226.891)
3780,Halve Maan-Noord,Utrecht,9336,1735,830,900,870,25,19,6,"POLYGON ((133770.466 456647.956, 133786.743 45...",POINT (134019.414 456085.584)
3781,Lombok-Oost,Utrecht,17630,2285,1110,1175,1470,13,13,0,"POLYGON ((135822.953 456037.558, 135715.534 45...",POINT (135529.769 455965.237)
...,...,...,...,...,...,...,...,...,...,...,...,...
3883,Veldhuizen,Utrecht,5631,9330,4635,4690,3460,189,166,23,"POLYGON ((129692.900 455386.278, 129786.270 45...",POINT (129497.100 454808.760)
3884,De Meern-Noord,Utrecht,3925,5735,2875,2860,2080,148,146,2,"POLYGON ((130301.248 456100.621, 130314.591 45...",POINT (130805.664 455491.330)
3885,De Meern-Zuid,Utrecht,1671,6425,3160,3260,2715,389,385,5,"POLYGON ((130607.044 455121.579, 130987.916 45...",POINT (129878.296 453978.297)
3886,Bedrijvengebied Oudenrijn,Utrecht,170,265,125,140,115,163,158,5,"POLYGON ((132114.247 455104.252, 132112.341 45...",POINT (131857.283 454254.670)


## Step 3: Spatial Join met Snuffelfiets datapunten

In [6]:
df = pd.read_csv("../data/external/city/resource_2020_06_15_2020_06_22.csv")
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'],df['lat'], crs="EPSG:4326"))
geo_df = geo_df.to_crs("EPSG:28992")

geo_df = gpd.sjoin(geo_df, utrecht_neigh, how="inner", op='within')

geo_df

ValueError: 'index_left' and 'index_right' cannot be names in the frames being joined

## Step 4: Median PM2.5 value per neighborhood, per day, per hour = timeseries per neighborhood

In [128]:
geo_df['recording_time'] = pd.to_datetime(df['recording_time'], format="%Y-%m-%d %H:%M:%S")
geo_df['date'] = geo_df['recording_time'].dt.date
geo_df['hour'] = geo_df['recording_time'].dt.hour

In [129]:
test = geo_df[["BU_NAAM", "date", "hour", "pm2_5"]].groupby(["BU_NAAM", "date", "hour"]).median().reset_index()

In [82]:
df = test[["BU_NAAM", "date", "pm2_5"]].groupby(["BU_NAAM"]).count().reset_index().sort_values(by=['pm2_5'], ascending=False)

fig = px.bar(df, x="BU_NAAM", y="pm2_5", title="Total recorded activity per neighborhood", )
fig.show()

In [78]:
df = test[["BU_NAAM", "date", "pm2_5"]].groupby(["BU_NAAM", "date"]).count().reset_index()

fig = px.bar(df, x='date', y='pm2_5', color="BU_NAAM")
fig.show()

In [85]:
test["datetime"] = pd.to_datetime(test.date) + test.hour.astype('timedelta64[h]')

In [86]:
test.info()

,BU_NAAM,date,hour,pm2_5,datetime
0,2e Daalsebuurt en omgeving,2020-06-15,15,4.0,2020-06-15 15:00:00
1,2e Daalsebuurt en omgeving,2020-06-15,16,2.0,2020-06-15 16:00:00
2,2e Daalsebuurt en omgeving,2020-06-15,17,3.0,2020-06-15 17:00:00
3,2e Daalsebuurt en omgeving,2020-06-17,13,3.0,2020-06-17 13:00:00
4,2e Daalsebuurt en omgeving,2020-06-17,18,6.0,2020-06-17 18:00:00
...,...,...,...,...,...
2346,Zuilen-Noord,2020-06-20,16,3.0,2020-06-20 16:00:00
2347,Zuilen-Noord,2020-06-20,17,3.0,2020-06-20 17:00:00
2348,Zuilen-Noord,2020-06-20,20,1.0,2020-06-20 20:00:00
2349,Zuilen-Noord,2020-06-21,12,2.0,2020-06-21 12:00:00


In [95]:
test.groupby(["BU_NAAM"]).count().reset_index()

,BU_NAAM,date,hour,pm2_5,datetime
0,2e Daalsebuurt en omgeving,13,13,13,13
1,"Abstede, Tolsteegsingel e.o.",22,22,22,22
2,Bedrijvengebied Kanaleneiland,39,39,39,39
3,Bedrijvengebied Oudenrijn,14,14,14,14
4,Bedrijvengebied Papendorp,31,31,31,31
...,...,...,...,...,...
105,Wolga- en Donaudreef en omgeving,19,19,19,19
106,Zambesidreef en omgeving,10,10,10,10
107,Zamenhofdreef en omgeving,24,24,24,24
108,"Zeeheldenbuurt, Hengeveldstraat en omgeving",43,43,43,43


In [98]:
fig = px.bar(test.groupby(["datetime"]).count().reset_index(), x='datetime', y='pm2_5', title='Measurements per hour', )
fig.show()

In [122]:
fig = px.bar(test.groupby(["BU_NAAM", "datetime"]).count().reset_index(), x='datetime', y='pm2_5', title='Measurements per hour', color="BU_NAAM")
fig.show()

In [108]:
export = test.merge(utrecht_neigh[["BU_NAAM", "geometry", "measurement_station"]], on="BU_NAAM")

In [109]:
export

,BU_NAAM,date,hour,pm2_5,datetime,geometry,measurement_station
0,2e Daalsebuurt en omgeving,2020-06-15,15,4.0,2020-06-15 15:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720)
1,2e Daalsebuurt en omgeving,2020-06-15,16,2.0,2020-06-15 16:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720)
2,2e Daalsebuurt en omgeving,2020-06-15,17,3.0,2020-06-15 17:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720)
3,2e Daalsebuurt en omgeving,2020-06-17,13,3.0,2020-06-17 13:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720)
4,2e Daalsebuurt en omgeving,2020-06-17,18,6.0,2020-06-17 18:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720)
...,...,...,...,...,...,...,...
2346,Zuilen-Noord,2020-06-20,16,3.0,2020-06-20 16:00:00,"POLYGON ((133293.957 459931.701, 133319.133 45...",POINT (133259.666 459263.855)
2347,Zuilen-Noord,2020-06-20,17,3.0,2020-06-20 17:00:00,"POLYGON ((133293.957 459931.701, 133319.133 45...",POINT (133259.666 459263.855)
2348,Zuilen-Noord,2020-06-20,20,1.0,2020-06-20 20:00:00,"POLYGON ((133293.957 459931.701, 133319.133 45...",POINT (133259.666 459263.855)
2349,Zuilen-Noord,2020-06-21,12,2.0,2020-06-21 12:00:00,"POLYGON ((133293.957 459931.701, 133319.133 45...",POINT (133259.666 459263.855)


In [123]:
export['lon'] = export.measurement_station.apply(lambda p: p.x)
export['lat'] = export.measurement_station.apply(lambda p: p.y)

export.to_csv("../data/interim/export_2020_06_15_2020_06_22_hourly.csv",index=False)

In [120]:
export

,BU_NAAM,date,hour,pm2_5,datetime,geometry,measurement_station,lon,lat
0,2e Daalsebuurt en omgeving,2020-06-15,15,4.0,2020-06-15 15:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720),135343.784673,456782.7200
1,2e Daalsebuurt en omgeving,2020-06-15,16,2.0,2020-06-15 16:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720),135343.784673,456782.7200
2,2e Daalsebuurt en omgeving,2020-06-15,17,3.0,2020-06-15 17:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720),135343.784673,456782.7200
3,2e Daalsebuurt en omgeving,2020-06-17,13,3.0,2020-06-17 13:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720),135343.784673,456782.7200
4,2e Daalsebuurt en omgeving,2020-06-17,18,6.0,2020-06-17 18:00:00,"POLYGON ((135456.341 457158.281, 135469.700 45...",POINT (135343.785 456782.720),135343.784673,456782.7200
...,...,...,...,...,...,...,...,...,...
2346,Zuilen-Noord,2020-06-20,16,3.0,2020-06-20 16:00:00,"POLYGON ((133293.957 459931.701, 133319.133 45...",POINT (133259.666 459263.855),133259.665771,459263.8551
2347,Zuilen-Noord,2020-06-20,17,3.0,2020-06-20 17:00:00,"POLYGON ((133293.957 459931.701, 133319.133 45...",POINT (133259.666 459263.855),133259.665771,459263.8551
2348,Zuilen-Noord,2020-06-20,20,1.0,2020-06-20 20:00:00,"POLYGON ((133293.957 459931.701, 133319.133 45...",POINT (133259.666 459263.855),133259.665771,459263.8551
2349,Zuilen-Noord,2020-06-21,12,2.0,2020-06-21 12:00:00,"POLYGON ((133293.957 459931.701, 133319.133 45...",POINT (133259.666 459263.855),133259.665771,459263.8551


## Same, but daily

In [132]:
test2 = geo_df[["BU_NAAM", "date", "hour", "pm2_5"]].groupby(["BU_NAAM", "date"]).median().reset_index()
export = test2.merge(utrecht_neigh[["BU_NAAM", "geometry", "measurement_station"]], on="BU_NAAM")

export['lon'] = export.measurement_station.apply(lambda p: p.x)
export['lat'] = export.measurement_station.apply(lambda p: p.y)

export.to_csv("../data/interim/export_2020_06_15_2020_06_22_daily.csv",index=False)